In [1]:
import os
os.chdir("../..")

In [2]:

import nest_asyncio
import asyncio
nest_asyncio.apply()

from uuid import uuid4
from agents import Agent, Runner, function_tool

In [3]:

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
result

RunResult(input='Write a haiku about recursion in programming.', new_items=[MessageOutputItem(agent=Agent(name='Assistant', instructions='You are a helpful assistant', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again'), raw_item=ResponseOutputMessage(id='msg_67df094a40788190913161439deed60e0e73188225dadcb6', content=[ResponseOutputText(annotations=[], text='Code loops endlessly,  \nFunctions call themselves again—  \nInfinite in scope.', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')], raw_responses=[ModelResponse(output=[ResponseOutputMessage(id='msg_67df094a40788190913161439deed60e0e73188225dadcb6', content=[ResponseOutputText(a

In [4]:
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)



await Runner.run(triage_agent, input="Hola, ¿cómo estás?")



RunResult(input='Hola, ¿cómo estás?', new_items=[HandoffCallItem(agent=Agent(name='Triage agent', instructions='Handoff to the appropriate agent based on the language of the request.', handoff_description=None, handoffs=[Agent(name='Spanish agent', instructions='You only speak Spanish.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again'), Agent(name='English agent', instructions='You only speak English', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[]

In [5]:
from pydantic import BaseModel

class Code(BaseModel):
    code: str
    

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

@function_tool
def visit_despegar():
    # Configurar opciones del navegador
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ejecuta el navegador en modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # Ruta al chromedriver
    service = Service('/usr/local/bin/chromedriver')  # Reemplaza con la ruta a tu chromedriver

    # Inicializar el navegador
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # Navegar a Despegar
        driver.get("https://www.despegar.com.ar")

        # Esperar a que la página cargue completamente
        time.sleep(5)  # Espera estática de 5 segundos; puedes usar WebDriverWait para una espera más dinámica

        # Tomar una captura de pantalla
        screenshot_path = "despegar_screenshot.png"
        driver.save_screenshot(screenshot_path)

        # Obtener el contenido de la página
        content = driver.page_source

        return f"Página Despegar visitada con éxito. Captura guardada en {screenshot_path}. Longitud del HTML: {len(content)} caracteres."

    except Exception as e:
        return f"Se produjo un error al visitar Despegar: {e}"

    finally:
        # Cerrar el navegador
        driver.quit()


In [ ]:
from playwright.async_api import async_playwright

async def visit_despegar_async() -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://www.google.com.ar")
        await page.wait_for_load_state("networkidle")
        screenshot_path = "despegar_screenshot.png"
        await page.screenshot(path=screenshot_path)
        content = await page.content()
        await browser.close()
        return f"Página Despegar visitada con éxito. Captura guardada en {screenshot_path}. Longitud del HTML: {len(content)} caracteres."

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"


In [26]:
await visit_despegar_async()

'Página Despegar visitada con éxito. Captura guardada en despegar_screenshot.png. Longitud del HTML: 235335 caracteres.'

In [9]:
programming_agent = Agent(
    name="Weather agent",
    instructions="You are a weather agent, if the weather is sunny, go to Despegar.com.ar",
    model="gpt-4o-mini",
    tools=[visit_despegar, get_weather]
)

In [10]:
result = await Runner.run(programming_agent, input="give the weather in Rosario, Santa Fe. If the weather is good, go to Despegar")

In [11]:
result.final_output

'The weather in Rosario, Santa Fe is sunny! However, it seems there was an issue trying to access Despegar.com.ar. You might want to check the website directly for travel options.'

In [12]:

programming_agent = Agent(
    name="Programmer agent",
    instructions="You are specialist in programming with python",
    model="gpt-4o-mini"
)

math_agent = Agent(
    name="Math agent",
    instructions="You are an expert in math and logic thinking",
    model="o3-mini"
)

senior_agent = Agent(
    name="Senior agent",
    instructions="You are a senior progammer who works in a multinational company. You have to check code and test quality",
    model="gpt-4"
)

orchestrator_agent = Agent(
    name="Project Manager",
    instructions="You are project manager who decides who have to work in the project",
    model="gpt-4o",
    handoffs=[programming_agent, math_agent, senior_agent],
    handoff_description="You can use 3 agents, a senior agent who is a qa, A math agent that you can use for the logic, and a programming agent that programs",
    output_type=Code
)


In [13]:
result = await Runner.run(orchestrator_agent, input="Create a project where i can send a number and i will gent the double of the number i send")

In [14]:
result.final_output

'Sure! Here\'s a simple Python project that allows you to send a number and get back its double. You can create a command-line application or use a simple web application. Below, I’ll provide an example of both approaches.\n\n### Command-Line Application\n\n1. **Create a file named `double_number.py`.**\n2. **Add the following code:**\n\n```python\ndef double_number(number):\n    return number * 2\n\ndef main():\n    try:\n        user_input = float(input("Enter a number: "))\n        result = double_number(user_input)\n        print(f"The double of {user_input} is {result}.")\n    except ValueError:\n        print("Please enter a valid number.")\n\nif __name__ == "__main__":\n    main()\n```\n\n3. **Run the application:**\n   - Open your terminal.\n   - Navigate to the directory where `double_number.py` is located.\n   - Run the script using the command: `python double_number.py`.\n\n### Web Application (Using Flask)\n\n1. **Install Flask if you haven\'t already.** You can do that usi